In [2]:
from google.colab import files
uploaded = files.upload()

Saving Articles.csv to Articles.csv


In [3]:
!pip install -q pyLDAvis

     |████████████████████████████████| 1.6MB 2.4MB/s 


In [7]:
import io
import pandas as pd
import numpy as np
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv(io.BytesIO(uploaded['Articles.csv']), encoding = 'unicode_escape')
df.head()

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business


In [8]:
df1 = df[['Article']]
df1.head()

,Article
0,KARACHI: The Sindh government has decided to b...
1,HONG KONG: Asian markets started 2015 on an up...
2,HONG KONG: Hong Kong shares opened 0.66 perce...
3,HONG KONG: Asian markets tumbled Tuesday follo...
4,NEW YORK: US oil prices Monday slipped below $...


PorterStemmer uses Suffix Stripping to produce stems. Notice how the PorterStemmer is giving the root (stem) of the word "cats" by simply removing the 's' after cat. This is a suffix added to cat to make it plural. PorterStemmer algorithm does not follow linguistics rather a set of 05 rules for different cases that are applied in phases (step by step) to generate stems his is the reason why PorterStemmer does not often generate stems that are actual English words. It does not keep a lookup table for actual stems of the word but applies algorithmic rules to generate stems. It uses the rules to decide whether it is wise to strip a suffix.

SnowballStemmers is used to create non-English Stemmers!

The LancasterStemmer (Paice-Husk stemmer) is an iterative algorithm with rules saved externally. One table containing about 120 rules indexed by the last letter of a suffix. On each iteration, it tries to find an applicable rule by the last character of the word. Each rule specifies either a deletion or replacement of an ending. If there is no such rule, it terminates. It also terminates if a word starts with a vowel and there are only two letters left or if a word starts with a consonant and there are only three characters left. Otherwise, the rule is applied, and the process repeats. LancasterStemmer is simple, but heavy stemming due to iterations and over-stemming may occur. Over-stemming causes the stems to be not linguistic, or they may have no meaning

In [9]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')


porter = PorterStemmer()
def lemmatize_stemming(text):
    return porter.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

processed_docs = df1['Article'].map(preprocess)
processed_docs[:10]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


0    [karachi, sindh, govern, decid, bring, public,...
1    [hong, kong, asian, market, start, upsw, limit...
2    [hong, kong, hong, kong, share, open, percent,...
3    [hong, kong, asian, market, tumbl, tuesday, fo...
4    [york, price, monday, slip, barrel, time, year...
5    [york, price, tumbl, tuesday, fresh, year, low...
6    [karachi, strong, bull, friday, pull, benchmar...
7    [singapor, fell, asia, monday, weak, demand, s...
8    [karachi, wholesal, market, rat, sugar, drop, ...
9    [sydney, price, fell, percent, wednesday, worl...
Name: Article, dtype: object

Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation. Words that have fewer than 3 characters are removed. All stopwords are removed. Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present. Words are stemmed — words are reduced to their root form.

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(processed_docs))

print(data_words[:10])

[['karachi', 'sindh', 'govern', 'decid', 'bring', 'public', 'transport', 'fare', 'cent', 'massiv', 'reduct', 'petroleum', 'product', 'price', 'feder', 'govern', 'news', 'report', 'sourc', 'say', 'reduct', 'fare', 'applic', 'public', 'transport', 'rickshaw', 'taxi', 'mean', 'travel', 'karachi', 'transport', 'ittehad', 'refus', 'abid', 'govern', 'decis', 'presid', 'irshad', 'bukhari', 'say', 'commut', 'charg', 'lowest', 'fare', 'karachi', 'compar', 'part', 'countri', 'add', 'vehicl', 'compress', 'natur', 'bukhari', 'say', 'karachi', 'transport', 'fare', 'decreas', 'price'], ['hong', 'kong', 'asian', 'market', 'start', 'upsw', 'limit', 'trade', 'friday', 'mainland', 'chines', 'stock', 'surg', 'hong', 'kong', 'specul', 'beij', 'eas', 'monetari', 'polici', 'boost', 'slow', 'growth', 'hong', 'kong', 'rise', 'percent', 'close', 'point', 'higher', 'seoul', 'close', 'percent', 'rise', 'point', 'sydney', 'gain', 'percent', 'point', 'close', 'singapor', 'edg', 'percent', 'gain', 'point', 'market'

In [11]:
# de-tokenization
detokenized_doc = []
for i in range(len(data_words)):
    seperator = ','
    t = ' '.join(data_words[i])
    detokenized_doc.append(t)
    
data_words = detokenized_doc
data_words[:10]

['karachi sindh govern decid bring public transport fare cent massiv reduct petroleum product price feder govern news report sourc say reduct fare applic public transport rickshaw taxi mean travel karachi transport ittehad refus abid govern decis presid irshad bukhari say commut charg lowest fare karachi compar part countri add vehicl compress natur bukhari say karachi transport fare decreas price',
 'hong kong asian market start upsw limit trade friday mainland chines stock surg hong kong specul beij eas monetari polici boost slow growth hong kong rise percent close point higher seoul close percent rise point sydney gain percent point close singapor edg percent gain point market mainland china japan taiwan zealand philippin thailand remain close holiday mainland bours shut januari share chines develop financi compani surg hong kong stoke hop beij eas monetari polici support lag growth world second largest economi china vank countri biggest develop sale leap percent peopl insur compani

The LDA topic model algorithm requires a document word matrix as the main input.
The CountVectorizer is configured to consider words that has occurred at least 10 times (min_df), remove built-in english stopwords, convert all words to lowercase, and a word can contain numbers and alphabets of at least length 3 in order to be qualified as a word.
Since most cells contain zeros, the result will be in the form of a sparse matrix to save memory.
Call the todense() method of the sparse matrix to materialize it to 2D array.

In [12]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_words)
data_vectorized

<2692x3746 sparse matrix of type '<class 'numpy.int64'>'
	with 245723 stored elements in Compressed Sparse Row format>

In [13]:
data_dense = data_vectorized.todense()
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  2.4367051452207766 %


learning_method : ‘batch’ | ‘online’, default=’batch’
Method used to update _component. Only used in fit method. In general, if the data size is large, the online update will be much faster than the batch update.

Valid options:

'batch': Batch variational Bayes method. Use all training data in
    each EM update.
    Old `components_` will be overwritten in each iteration.
'online': Online variational Bayes method. In each EM update, use
    mini-batch of training data to update the ``components_``
    variable incrementally. The learning rate is controlled by the
    ``learning_decay`` and the ``learning_offset`` parameters.
    
    
 evaluate_every : int, optional (default=0)
How often to evaluate perplexity. Only used in fit method. set it to 0 or negative number to not evalute perplexity in training at all. Evaluating perplexity can help you check convergence in training process, but it will also increase total training time. Evaluating perplexity in every iteration might increase training time up to two-fold.

random_state : int, RandomState instance or None, optional (default=None)
If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.

learning_offset : float, optional (default=10.)
A (positive) parameter that downweights early iterations in online learning. It should be greater than 1.0. In the literature, this is called tau_0.

learning_decay : float, optional (default=0.7)
It is a parameter that control learning rate in the online learning method. The value should be set between (0.5, 1.0] to guarantee asymptotic convergence. When the value is 0.0 and batch_size is n_samples, the update method is same as batch learning. In the literature, this is called kappa.

In [14]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size= 100,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                      learning_decay = 0.9              
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)
lda_output

LatentDirichletAllocation(batch_size=100, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.9,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


array([[2.42214008e-01, 1.96098829e-03, 2.45372037e-01, ...,
        1.96109762e-03, 1.96107267e-03, 1.96121696e-03],
       [2.44076215e-02, 2.50685296e-04, 2.50658725e-04, ...,
        2.50675636e-04, 8.79656397e-01, 9.41811939e-02],
       [4.34803709e-03, 4.34904600e-03, 4.34848611e-03, ...,
        4.34804799e-03, 9.60865945e-01, 4.34833251e-03],
       ...,
       [7.35368632e-04, 1.14772511e-01, 2.96481673e-01, ...,
        7.35459502e-04, 7.35537594e-04, 7.35426535e-04],
       [7.75440835e-04, 7.75305537e-04, 7.75291715e-04, ...,
        7.75348149e-04, 1.31506832e-01, 7.75407764e-04],
       [3.76009772e-04, 1.82799902e-01, 2.50657788e-02, ...,
        3.76048191e-04, 3.76015208e-04, 3.76002049e-04]])

In statistics, maximum likelihood estimation is a method of estimating the parameters of a statistical model, given observations. The method obtains the parameter estimates by finding the parameter values that maximize the likelihood function.

n information theory, perplexity is a measurement of how well a probability distribution or probability model predicts a sample. It may be used to compare probability models. A low perplexity indicates the probability distribution is good at predicting the sample.

A model with higher log-likelihood and lower perplexity (exp(-1. * log-likelihood per word)) is considered to be good.

On a different note, perplexity might not be the best measure to evaluate topic models because it doesn’t consider the context and semantic associations between words. This can be captured using topic coherence measure

In [15]:
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2497977.6845504693
Perplexity:  957.3053109103854
{'batch_size': 100,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.9,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


The most important tuning parameter for LDA models is n_components (number of topics). In addition, I am going to search learning_decay (which controls the learning rate) as well.

Besides these, other possible search params could be learning_offset (downweigh early iterations. Should be > 1) and max_iter. 

The grid search constructs multiple LDA models for all possible combinations of param values in the param_grid dict.

To classify a document as belonging to a particular topic, a logical approach is to see which topic has the highest contribution to that document and assign it.

In [16]:
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(df1))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.24,0,0.25,0,0,0.5,0,0,0,0,5
Doc1,0.02,0,0,0,0,0,0,0,0.88,0.09,8
Doc2,0,0,0,0,0,0,0,0,0.96,0,8
Doc3,0,0,0.01,0,0,0,0,0,0.77,0.21,8
Doc4,0.02,0.01,0,0,0,0,0.02,0,0.25,0.7,9
Doc5,0,0,0,0,0,0,0,0.32,0.1,0.58,9
Doc6,0,0.08,0,0.07,0,0,0,0,0.83,0,8
Doc7,0,0,0,0,0,0,0,0,0,0.99,9
Doc8,0,0,0,0,0,0,0,0.89,0.09,0,7
Doc9,0,0,0,0,0,0,0,0.11,0.19,0.69,9


In [17]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,4,586
1,1,542
2,6,402
3,8,319
4,0,262
5,3,247
6,9,230
7,2,45
8,5,43
9,7,16


The pyLDAvis offers the best visualization to view the topics-keywords distribution.

A good topic model will have non-overlapping, fairly big sized blobs for each topic.

pyLDAvis.prepare(topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency, R=30, lambda_step=0.01, mds=<function js_PCoA>, n_jobs=-1, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, sort_topics=True)[source]
Transforms the topic model distributions and related corpus data into the data structures needed for the visualization.
  
topic_term_dists:array-like, shape (n_topics, n_terms)
Matrix of topic-term probabilities. Where n_terms is len(vocab).

doc_topic_dists :array-like, shape (n_docs, n_topics)
Matrix of document-topic probabilities.
  
 mds :function or a string representation of function
A function that takes topic_term_dists as an input and outputs a n_topics by 2 distance matrix. The output approximates the distance between topics.
  
  

In [18]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x          y  topics  cluster       Freq
topic                                                   
1      138.711273  52.746719       1        1  18.233161
4      135.366440 -30.453398       2        1  18.106107
8       90.719398 -98.281944       3        1  16.302521
6       18.542175   8.959215       4        1  15.663326
9       59.851025 -40.444321       5        1  10.571751
0      -41.810856 -19.654461       6        1   8.200244
3       82.097038  20.046312       7        1   7.415189
2       57.498611  84.566566       8        1   3.350964
7      -17.406153  64.825760       9        1   1.427873
5        4.773799 -80.313843      10        1   0.728864, topic_info=     Category         Freq          Term        Total  loglift  logprob
2477  Default  1869.000000         price  1869.000000  30.0000  30.0000
2297  Default  3256.000000      pakistan  3256.000000  29.0000  29.0000
2354  Default  3258.000000       percent  3258.000000  28.0000  28.0000
2838  Default  6349.000000           say  6349.000000  27.0000  27.0000
1965  Default  1865.000000        market  1865.000000  26.0000  26.0000
2498  Default   934.000000       product   934.000000  25.0000  25.0000
779   Default  1581.000000       cricket  1581.000000  24.0000  24.0000
3660  Default  1333.000000        wicket  1333.000000  23.0000  23.0000
353   Default  1205.000000       billion  1205.000000  22.0000  22.0000
1588  Default   712.000000       increas   712.000000  21.0000  21.0000
3323  Default  1624.000000          team  1624.000000  20.0000  20.0000
794   Default  1017.000000         crude  1017.000000  19.0000  19.0000
1075  Default  1544.000000       england  1544.000000  18.0000  18.0000
280   Default  1269.000000          ball  1269.000000  17.0000  17.0000
2049  Default  1483.000000       million  1483.000000  16.0000  16.0000
301   Default   838.000000        barrel   838.000000  15.0000  15.0000
1391  Default  1002.000000        govern  1002.000000  14.0000  14.0000
3356  Default  1688.000000          test  1688.000000  13.0000  13.0000
2091  Default  1376.000000         month  1376.000000  12.0000  12.0000
3716  Default  3772.000000          year  3772.000000  11.0000  11.0000
2836  Default   420.000000         saudi   420.000000  10.0000  10.0000
1670  Default   597.000000     islamabad   597.000000   9.0000   9.0000
1501  Default   696.000000          high   696.000000   8.0000   8.0000
1982  Default  1669.000000         match  1669.000000   7.0000   7.0000
291   Default  1178.000000          bank  1178.000000   6.0000   6.0000
2056  Default   913.000000        minist   913.000000   5.0000   5.0000
123   Default   523.000000          amir   523.000000   4.0000   4.0000
1375  Default   892.000000        global   892.000000   3.0000   3.0000
2233  Default   556.000000          opec   556.000000   2.0000   2.0000
2791  Default   887.000000           run   887.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
2632  Topic10    32.166019     recommend    91.042351   3.8810  -4.4123
2375  Topic10    91.440885     petroleum   275.330640   3.8192  -3.3676
2598  Topic10     3.911231       ratchet    12.121962   3.7903  -6.5194
851   Topic10    25.164138       decreas    94.640121   3.5968  -4.6578
3088  Topic10    19.827245         speed    79.230040   3.5361  -4.8962
2560  Topic10     3.724808         queue    15.379442   3.5034  -6.5682
1867  Topic10    25.984643         light   121.423110   3.3797  -4.6257
3013  Topic10    10.886983         slash    59.137033   3.2291  -5.4957
1620  Topic10     4.368311       inspect    25.030210   3.1757  -6.4089
3499  Topic10    16.555086       unchang    97.479801   3.1485  -5.0766
3192  Topic10     7.677666  strongaccord    45.391725   3.1444  -5.8449
2281  Topic10    10.278194         owner    61.414851   3.1338  -5.5532
248   Topic10    39.608825        author   248.909721   3.0834  -4.2042
1315  Topic10     2.484200       

The weights of each keyword in each topic is contained in lda_model.components_ as a 2d array. The names of the keywords itself can be obtained from vectorizer object using get_feature_names().

In [22]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,aaron,abandon,abbasi,abbott,abdul,abdullah,abil,abl,abroad,absenc,absolut,absorb,abus,academi,acceler,accept,access,accid,accommod,accompani,accord,account,accumul,accur,accus,ace,achiev,acknowledg,acquir,acquisit,act,action,activ,actor,actual,adam,adapt,add,addit,address,...,worth,wouldn,wrap,wrist,write,wrong,xinjiang,yadav,yard,yasir,year,yellen,yellow,yemen,yesterday,yield,york,yorker,yorkshir,young,younger,youngest,youngster,youni,younu,yousuf,youth,yuan,yuvraj,zadran,zafar,zalmi,zaman,zampa,zealand,zero,zimbabw,zone,zulfiqar,zurich
Topic0,0.109129,0.433318,14.658335,0.106340,1.015278,2.898750,0.223263,0.547120,0.159709,1.840619,0.663483,0.142453,0.110572,0.107234,9.594279,8.698373,3.907012,0.106844,3.913924,0.126957,120.051575,55.953455,9.570526,0.105537,0.109054,0.104887,60.953338,0.120355,0.124950,0.667009,0.112858,4.437871,10.011064,0.106127,9.759138,0.112297,0.113877,89.967160,50.426714,44.482946,...,78.231859,0.106581,0.106542,0.106077,0.112389,0.122391,2.865503,0.105232,0.106149,0.106417,714.090023,0.111716,1.371645,0.109428,0.107009,10.693314,0.443624,0.106356,0.105071,0.107520,0.107448,0.107020,0.106779,0.106497,0.105863,0.150507,2.919059,0.109076,0.106387,0.105070,0.109609,0.132426,0.105285,0.106865,0.109412,2.686808,0.106064,0.107277,0.106931,0.111359
Topic1,4.119602,3.530102,0.212587,0.125952,0.152375,0.121139,12.420458,66.612844,3.873115,15.420269,20.098752,1.116690,5.603360,16.653784,0.296459,22.107747,0.155488,3.739605,0.166036,2.525956,22.521421,5.971363,2.116130,0.132618,10.980206,14.240673,37.773709,13.727464,0.127695,0.106096,6.964569,54.031986,3.313789,11.215022,14.178068,4.913718,8.895966,160.159479,2.163403,0.366746,...,0.130477,17.740729,11.337923,29.431086,33.179451,23.312092,0.105446,10.632095,15.308928,1.058409,1151.639437,0.106195,22.178229,0.472343,9.079425,0.123387,14.842912,0.192501,0.219865,69.876468,12.097524,15.284541,13.312136,0.425689,0.111252,0.106754,14.404184,0.107881,1.773759,0.131192,0.106662,0.195334,0.127671,0.133808,17.233747,8.271848,1.064723,0.122164,0.125380,20.041710
Topic2,2.848978,0.593771,0.106511,0.106880,0.112486,0.108360,0.114519,2.998725,0.110982,1.949682,0.298362,0.335582,7.187341,0.113941,1.208995,0.143722,0.109018,15.471590,0.112771,0.149692,12.872104,14.130251,0.109496,0.106273,14.673842,0.105096,0.106713,0.106912,4.907331,0.121632,12.065138,4.110734,1.412424,0.131577,2.107091,4.557271,0.110299,15.090404,0.224406,0.113081,...,6.341398,0.110545,0.110499,0.106661,0.108571,1.762980,0.105654,0.108856,0.194917,0.105872,35.140945,0.108905,0.511897,0.116434,0.112123,0.108749,5.448742,0.107124,0.106524,0.107006,0.109921,0.115794,0.122979,0.109245,0.106778,0.108122,0.107307,0.110762,0.107041,0.107084,0.108179,0.105025,0.109485,0.120263,4.501150,0.111821,0.138421,0.108987,0.106206,0.116142
Topic3,0.110767,1.234547,0.191694,0.106499,8.909751,0.115769,0.113258,0.235853,0.386120,0.142817,0.287408,0.795815,0.599409,24.716429,0.229451,21.348295,0.252279,0.111978,1.151727,2.806516,44.692529,3.099380,0.132200,0.109427,6.045810,0.105968,19.157933,0.350405,0.128289,0.319318,6.627637,40.793460,10.562868,0.188455,0.446936,0.117872,0.114031,97.615253,3.606798,8.494159,...,3.058952,0.356169,0.112584,0.127909,37.646605,0.185082,0.105627,0.108626,0.113641,91.486042,103.521431,0.147957,0.107991,0.107045,0.126051,0.142348,0.146178,0.106156,0.107386,14.623994,0.107656,0.117797,8.098193,93.553797,16.049524,6.974438,3.548694,0.106283,0.108271,0.149531,14.697957,25.535855,16.028856,0.104891,16.326886,1.572693,0.108028,0.191298,23.027337,0.116491
Topic4,42.999503,14.996241,0.132448,27.952119,0.110174,0.104982,8.200564,8.250470,0.106186,25.247915,1.197156,0.108277,1.478067,0.372232,6.807447,0.189264,0.110473,0.106050,2.050671,1.692398,0.469035,6.998822,4.366930,16.024795,0.127709,2.219758,26.664038,0.138924,0.105279,0.106819,0.111393,26.952604,0.128737,1.101744,0.111188,66.464410,3.711857,165.640201,10.671716,0.113113,...,7.343278,0.182193,18.438698,4.510986,0.453084,3.369088,0.106688,16.909844,0

In [23]:
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=15)        

df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,say,billion,year,million,govern,pakistan,minist,percent,countri,bank,sector,financ,export,import,increas
Topic 1,year,say,world,time,game,player,play,team,final,strong,cricket,match,open,second,tournament
Topic 2,euro,ronaldo,portug,say,unit,fifa,citi,investig,polic,real,footbal,madrid,turkey,million,messi
Topic 3,pakistan,say,cricket,team,amir,khan,board,mohammad,strong,karachi,islamabad,ahm,lahor,secur,nation
Topic 4,test,england,wicket,ball,match,run,captain,seri,play,bowl,lanka,second,score,australia,inning
Topic 5,price,diesel,litr,petroleum,petrol,increas,ogra,product,regulatori,author,islamabad,sourc,high,month,recommend
Topic 6,say,pakistan,countri,year,compani,strong,china,invest,world,includ,state,busi,billion,project,develop
Topic 7,saudi,product,market,cosmet,islam,say,arabia,sale,sell,korean,brand,king,year,beauti,global
Topic 8,percent,market,year,say,dollar,bank,rise,trade,week,stock,month,economi,index,point,gold
Topic 9,price,say,crude,barrel,market,opec,product,million,output,suppli,iran,year,week,cent,global


First the statement is preprocessed, followed by detokenization, vectorization and finally prediction.

In [28]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(text, nlp = nlp):
    global sent_to_words
    global lemmatization
    
    # Step 1: Clean with simple_preprocess and Lemmatize
    mytext_2 = preprocess(text)

    # Step 2: detokenize
    detokenized_doc = []
    for i in mytext_2:
      seperator = ','
      t = ' '.join(mytext_2)
      detokenized_doc.append(t)
      mytext_3 = detokenized_doc

    # Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)

    # Step 4: LDA Transform
    topic_probability_scores = lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = df1.loc[df1.index[4],'Article']
topic = predict_topic(text = mytext, nlp = nlp)
print(mytext)
print(topic)


NEW YORK: US oil prices Monday slipped below $50 a barrel for the first time in more than five years as the surging dollar and news of additional supplies extended a six-month rout.US benchmark West Texas Intermediate for February delivery, in free fall since June, ended at $50.04 a barrel, down $2.65 or five percent. The contract got as low as $49.95 a barrel earlier in the session, its lowest level since May 1, 2009.European benchmark Brent oil for February delivery fell $3.31 to $53.11 a barrel in London.Monday´s slide in oil prices followed indications of rising output from key producers Russia and Iraq at a time when forecasters have trimmed their demand projections due to weak global economic growth.The breaching of the psychologically important $50 level also came on a turbulent day for global financial markets. US stocks fell nearly two percent, approaching the drops in European equity markets as the euro plunged to a nine-year low on revived eurozone worries.A long rally in th